In [1]:
import pandas as pd
import os,time
import numpy as np
import nsepy
import datetime
from scipy.special import cbrt
import pickle

In [2]:
users = pickle.load(open('users.pickle','rb'))

In [3]:
os.chdir('..')

In [4]:
print('Hi Please enter your username or press n if you are a new user: ')
key = input()
if key == 'n' :
    print('Please type the username you want: ')
    key = input()
    users.append(key)
    os.mkdir(key)
    os.chdir(key)
else:
    flag = 0
    count = 0
    while flag == 0:
        if key in users:
            os.chdir(key)
            flag = 1
        else:
            count+=1
            if count==3:
                print("3 wrong attempts made.This progam is shutting down")
                break
            else:
                print("Wrong User name please ty again: ")
                key = input()
            

Hi Please enter your username or press n if you are a new user: 
n
Please type the username you want: 
Sahu


In [5]:
os.chdir('../codes/')

In [6]:
pickle.dump(users,open('users.pickle','wb'))

In [7]:
os.chdir('../'+key)

In [8]:
print("Hi {}. Welcome to your nifty investment assistant".format(key))
print("Let us know how many days you want to invest for: ")

flag = 0
while flag == 0:
    try:
        days = int(input())
        flag = 1
    except:
        print("""something went wrong please ty again""")


Hi Sahu. Welcome to your nifty investment assistant
Let us know how many days you want to invest for: 
30


In [9]:
print("Let us know how much more cash you want to invest if not press 0: ")

flag = 0
while flag == 0:
    try:
        cash = float(input())
        flag = 1
    except:
        print("""something went wrong please ty again""")

Let us know how much more cash you want to invest if not press 0: 
1000


In [10]:
end = datetime.date.today()
start = end - datetime.timedelta(days=days+10)

In [11]:
os.chdir('../input_data/')

In [12]:
companies = pd.read_csv('nifty_companies.csv',header=0,encoding='utf-8',engine='c',skip_blank_lines=True)


In [13]:
os.chdir('../'+key)

In [14]:
df = pd.DataFrame()
for company in list(companies['Symbol']):
    try:
        print(company+' is being collected')
        df_append = nsepy.get_history(company,start=start,end=end)
        df = df.append(df_append)
    except Exception as e:
        print(e)

ADANIPORTS is being collected
ASIANPAINT is being collected
AXISBANK is being collected
BAJAJ-AUTO is being collected
BAJFINANCE is being collected
BAJAJFINSV is being collected
BHARTIARTL is being collected
INFRATEL is being collected
BPCL is being collected
BRITANNIA is being collected
CIPLA is being collected
COALINDIA is being collected
DRREDDY is being collected
EICHERMOT is being collected
GAIL is being collected
GRASIM is being collected
HCLTECH is being collected
HDFC is being collected
HDFCBANK is being collected
HEROMOTOCO is being collected
HINDALCO is being collected
HINDUNILVR is being collected
ICICIBANK is being collected
IBULHSGFIN is being collected
INDUSINDBK is being collected
INFY is being collected
IOC is being collected
ITC is being collected
JSWSTEEL is being collected
KOTAKBANK is being collected
LT is being collected
M&M is being collected
MARUTI is being collected
NTPC is being collected
ONGC is being collected
POWERGRID is being collected
RELIANCE is being co

In [15]:
len(df['Symbol'].unique())

50

In [16]:
df.reset_index(inplace=True)

In [17]:
df = pd.merge(df,companies[['Symbol','Sector']],on='Symbol',how='left')

In [18]:
for index in ['',' auto',' bank',' fmcg',' it',' media',' metal',' pharma',' realty',' PSU BANK',' 500']:
    print('nifty {} is being merged'.format(index))
    to_merge = nsepy.get_history(str.upper('nifty'+index),start=start,end=end,index=True)
    to_merge.reset_index(inplace=True)
    print(to_merge['Date'].min(),to_merge['Date'].max())
    if index == ' PSU BANK':
        df = pd.merge(df,to_merge,on='Date',how='left',suffixes=['','_psu'])
    elif index=='':
        df = pd.merge(df,to_merge,on='Date',how='left',suffixes=['','_nifty'])
    else:
        df = pd.merge(df,to_merge,on='Date',how='left',suffixes=['','_'+str.strip(index)])

nifty  is being merged
2019-03-26 2019-05-03
nifty  auto is being merged
2019-03-26 2019-05-03
nifty  bank is being merged
2019-03-26 2019-05-03
nifty  fmcg is being merged
2019-03-26 2019-05-03
nifty  it is being merged
2019-03-26 2019-05-03
nifty  media is being merged
2019-03-26 2019-05-03
nifty  metal is being merged
2019-03-26 2019-05-03
nifty  pharma is being merged
2019-03-26 2019-05-03
nifty  realty is being merged
2019-03-26 2019-05-03
nifty  PSU BANK is being merged
2019-03-26 2019-05-03
nifty  500 is being merged
2019-03-26 2019-05-03


In [19]:
for col in df.columns:
    if df[col].isnull().any():
        print(col)

In [20]:
try:
    actual_portfolio = pd.read_excel('actual_portfolio.xlsx',header=0)
except:
    actual_portfolio = pd.DataFrame()
    actual_portfolio['Symbol'] = companies['Symbol']
    actual_portfolio['Money_invested'] = 0
    actual_portfolio['current value'] = 0
    actual_portfolio['shares'] = 0

In [21]:
investments = actual_portfolio.copy()

In [22]:
balance_sheet = pd.DataFrame(columns=['Date']+list(companies['Symbol'])+['cash','portfolio_value'])

In [23]:
investments.set_index('Symbol',inplace=True)

In [24]:
os.chdir('../input_data/')

In [25]:
price_report = pd.read_excel('price_report.xlsx',sheet_name='edd_v01',header=0)
prob_report =  pd.read_excel('prob_report.xlsx',sheet_name='edd_v01',header=0)
sectors = pd.read_csv('nifty_companies.csv',header=0,encoding='utf-8',engine='c',skip_blank_lines=True)
graham_stats = pd.read_excel('Graham_Stats.xlsx',header=0)

In [26]:
os.chdir('../price_pickles/')

In [27]:
price_numerical = pickle.load(open('numerical.pickle','rb'))
price_categorical = pickle.load(open('categorical.pickle','rb'))
price_conversion_dict = pickle.load(open('conversion_dict.pickle','rb'))
price_model_columns = pickle.load(open('model_columns','rb'))
price_lr_model_columns = pickle.load(open('lr_model_columns.pickle','rb'))
price_order = pickle.load(open('order.pickle','rb'))
price_pca = pickle.load(open('pca.pickle','rb'))
price_scale = pickle.load(open('scale.pickle','rb'))

In [28]:
os.chdir('../prob_pickles/')

In [29]:
prob_numerical = pickle.load(open('numerical.pickle','rb'))
prob_categorical = pickle.load(open('categorical.pickle','rb'))
prob_conversion_dict = pickle.load(open('conversion_dict.pickle','rb'))
prob_model_columns = pickle.load(open('model_columns','rb'))
prob_lr_model_columns = pickle.load(open('lr_model_columns.pickle','rb'))
prob_order = pickle.load(open('order.pickle','rb'))
prob_pca = pickle.load(open('pca.pickle','rb'))
prob_scale = pickle.load(open('scale.pickle','rb'))

In [30]:
os.chdir('../price_models/')

In [31]:
price_lr = pickle.load(open('LR.pickle','rb'))
price_gbm = pickle.load(open('GBM.pickle','rb'))
price_rf = pickle.load(open('RF.pickle','rb'))
price_rf_full = pickle.load(open('RF_full.pickle','rb'))
price_gbm_full = pickle.load(open('GBM_full.pickle','rb'))
price_rf_pca = pickle.load(open('RF_pca.pickle','rb'))
price_gbm_pca = pickle.load(open('GBM_pca.pickle','rb'))

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [32]:
os.chdir('../prob_models/')

In [33]:
prob_lr = pickle.load(open('LR.pickle','rb'))
prob_gbm = pickle.load(open('GBM.pickle','rb'))
prob_rf = pickle.load(open('RF.pickle','rb'))
prob_rf_full = pickle.load(open('RF_full.pickle','rb'))
prob_gbm_full = pickle.load(open('GBM_full.pickle','rb'))
prob_rf_pca = pickle.load(open('RF_pca.pickle','rb'))
prob_gbm_pca = pickle.load(open('GBM_pca.pickle','rb'))

In [34]:
corporations = [com for com in balance_sheet.columns if com not in ['Date','cash','portfolio_value']]

In [35]:
no_of_shares = {}
for company in investments.index:
    no_of_shares[company]=investments.loc[company,'shares']

In [36]:
actual_no_of_shares = no_of_shares.copy()

In [37]:
dates = sorted(df['Date'].unique())[-days//7*5:]

In [38]:
start_date = dates[0]
end_date = dates[-1]

In [39]:
transform_dict = {'log':lambda x: np.log(x),'sqr':lambda x: x**2,'sqrt':lambda x: np.sqrt(x),'exp':lambda x:np.exp(x),
                 'cube':lambda x: x**3,'cuberoot': lambda x: cbrt(x)}

In [40]:
#Creating Price Data
rename_dict = {col : '_'.join(col.split(' ')) for col in df.columns}
price_data = df.rename(rename_dict,copy=False,axis=1)
cols_to_shift = [col for col in price_data.columns if col not in ['Date','Symbol','Series','Prev_Close','Open','Sector','Close']]
rename_dict2 = {col : col+'_1d' for col in cols_to_shift}
price_dev = pd.DataFrame()
for company in corporations:
    print('development data for {}'.format(company))
    df_append = price_data[price_data['Symbol']==company]
    df_append['Date'] = pd.to_datetime(price_data['Date'])
    df_append.sort_values(by='Date',inplace=True)
    for index in ['nifty','auto','bank','fmcg','it','media','metal','pharma','realty','psu','500']:
        for i in range(1,6):
            df_append['return_{j}d_{index}'.format(j=i,index=index)] = (df_append['Close_{}'.format(index)].shift(i)-df_append['Open_{}'.format(index)].shift(i))/df_append['Open_{}'.format(index)].shift(i)*100
        df_append['Voltality_{}'.format(index)] = df_append[['return_{i}d_{index}'.format(i=i,index=index) for i in range(1,6)]].apply(lambda x:np.std(x),axis=1)
    for i in range(1,6):
        df_append['return_{j}d'.format(j=i)] = (df_append['Close'].shift(i)-df_append['Open'].shift(i))/df_append['Open'].shift(i)*100
    df_append['Voltality'] = df_append[['return_1d','return_2d','return_3d','return_4d','return_5d']].apply(lambda x:np.std(x),axis=1)
    df_append[['Open_1d']+cols_to_shift] = df_append[['Open']+cols_to_shift].shift(1)
    df_append.rename(rename_dict2,copy=False,inplace=True,axis=1)
    price_dev = price_dev.append(df_append.iloc[5:])
price_dev = price_dev[price_dev['Date'].isin(dates)]
rename_dict = {col : col.replace('%','pct_') for col in price_dev.columns if '%' in col}
price_dev.rename(rename_dict,axis=1,inplace=True)
rename_dict={}
for col in price_order.keys():
    price_dev[col]=price_dev[col]/10**price_order[col]
    rename_dict[col] = col+"(in {})".format("10^{}".format(price_order[col]))
price_dev.rename(rename_dict,axis=1,inplace=True)

for col in price_categorical:
    if len(price_conversion_dict[col])==1:
        category = price_conversion_dict[col][0]
        price_dev[col+'_dum_'+str(category)] = 0
        price_dev.loc[price_dev[col]==category,col+'_dum_'+str(category)]=1
    else:
        total_categories = len(price_conversion_dict[col])
        dummies = len(str(int(bin(total_categories)[2:],10)))
        bin_conv=[]
        for i in range(total_categories):
            bin_conv.append(price_conversion_dict[col][i][1])
        for j in range(dummies):
            price_dev[col+'_dum_'+str(j)]=0
            for i,cat in enumerate([conv[0] for conv in price_conversion_dict[col]]):
                price_dev.loc[price_dev[col]==cat,col+'_dum_'+str(j)]=bin_conv[i]%10
                bin_conv[i]=bin_conv[i]//10
    price_dev.drop(col,axis=1,inplace=True)

for col in list(price_report['Var']):
    applied = price_report.loc[price_report['Var']==col,'conversions'].values[0]
    if applied != '' and applied != 'categorical':
        try:
            for t in applied.split(','):
                price_dev[col+'_'+t] = price_dev[col].apply(transform_dict[t])
        except:
            pass
        
price_dev.reset_index(inplace=True,drop=True)
price_dev = price_dev[['Symbol','Date','Open','Close']+list(price_model_columns)]

development data for ADANIPORTS


C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://panda

development data for ASIANPAINT
development data for AXISBANK
development data for BAJAJ-AUTO
development data for BAJFINANCE
development data for BAJAJFINSV
development data for BHARTIARTL
development data for INFRATEL
development data for BPCL
development data for BRITANNIA
development data for CIPLA
development data for COALINDIA
development data for DRREDDY
development data for EICHERMOT
development data for GAIL
development data for GRASIM
development data for HCLTECH
development data for HDFC
development data for HDFCBANK
development data for HEROMOTOCO
development data for HINDALCO
development data for HINDUNILVR
development data for ICICIBANK
development data for IBULHSGFIN
development data for INDUSINDBK
development data for INFY
development data for IOC
development data for ITC
development data for JSWSTEEL
development data for KOTAKBANK
development data for LT
development data for M&M
development data for MARUTI
development data for NTPC
development data for ONGC
development

In [41]:
price_dev.isnull().any()

Symbol                            False
Date                              False
Open                              False
Close                             False
Prev_Close(in 10^5)               False
High_1d(in 10^5)                  False
Low_1d(in 10^5)                   False
Last_1d(in 10^5)                  False
VWAP_1d(in 10^5)                  False
Volume_1d(in 10^9)                False
Turnover_1d(in 10^11)             False
Trades_1d(in 10^7)                False
Deliverable_Volume_1d(in 10^9)    False
pct_Deliverble_1d                 False
Open_nifty_1d(in 10^5)            False
High_nifty_1d(in 10^5)            False
Low_nifty_1d(in 10^5)             False
Close_nifty_1d(in 10^5)           False
Volume_nifty_1d(in 10^9)          False
Turnover_nifty_1d(in 10^11)       False
Open_auto_1d(in 10^5)             False
High_auto_1d(in 10^5)             False
Low_auto_1d(in 10^5)              False
Close_auto_1d(in 10^5)            False
Open_bank_1d(in 10^5)             False


In [42]:
# prepaing probability data
rename_dict = {col : '_'.join(col.split(' ')) for col in df.columns}
prob_data = df.rename(rename_dict,copy=False,axis=1)
cols_to_shift = [col for col in prob_data.columns if col not in ['Date','Symbol','Series','Prev_Close','Open','Sector','Close']]
rename_dict2 = {col : col+'_1d' for col in cols_to_shift}
prob_dev = pd.DataFrame()
for company in corporations:
    print('development data for {}'.format(company))
    df_append = prob_data[prob_data['Symbol']==company]
    df_append['Date'] = pd.to_datetime(prob_data['Date'])
    df_append.sort_values(by='Date',inplace=True)
    for index in ['nifty','auto','bank','fmcg','it','media','metal','pharma','realty','psu','500']:
        for i in range(1,6):
            df_append['return_{j}d_{index}'.format(j=i,index=index)] = (df_append['Close_{}'.format(index)].shift(i)-df_append['Open_{}'.format(index)].shift(i))/df_append['Open_{}'.format(index)].shift(i)*100
        df_append['Voltality_{}'.format(index)] = df_append[['return_{i}d_{index}'.format(i=i,index=index) for i in range(1,6)]].apply(lambda x:np.std(x),axis=1)
    for i in range(1,6):
        df_append['return_{j}d'.format(j=i)] = (df_append['Close'].shift(i)-df_append['Open'].shift(i))/df_append['Open'].shift(i)*100
    df_append['Voltality'] = df_append[['return_1d','return_2d','return_3d','return_4d','return_5d']].apply(lambda x:np.std(x),axis=1)
    df_append[['Open_1d']+cols_to_shift] = df_append[['Open']+cols_to_shift].shift(1)
    df_append.rename(rename_dict2,copy=False,inplace=True,axis=1)
    prob_dev = prob_dev.append(df_append.iloc[5:])
prob_dev = prob_dev[prob_dev['Date'].isin(dates)]
rename_dict = {col : col.replace('%','pct_') for col in prob_dev.columns if '%' in col}
prob_dev.rename(rename_dict,axis=1,inplace=True)
rename_dict={}
for col in prob_order.keys():
    prob_dev[col]=prob_dev[col]/10**prob_order[col]
    rename_dict[col] = col+"_order{}".format(prob_order[col])
prob_dev.rename(rename_dict,axis=1,inplace=True)

for col in prob_categorical:
    if len(prob_conversion_dict[col])==1:
        category = prob_conversion_dict[col][0]
        prob_dev[col+'_dum_'+str(category)] = 0
        prob_dev.loc[prob_dev[col]==category,col+'_dum_'+str(category)]=1
    else:
        total_categories = len(prob_conversion_dict[col])
        dummies = len(str(int(bin(total_categories)[2:],10)))
        bin_conv=[]
        for i in range(total_categories):
            bin_conv.append(prob_conversion_dict[col][i][1])
        for j in range(dummies):
            prob_dev[col+'_dum_'+str(j)]=0
            for i,cat in enumerate([conv[0] for conv in prob_conversion_dict[col]]):
                prob_dev.loc[prob_dev[col]==cat,col+'_dum_'+str(j)]=bin_conv[i]%10
                bin_conv[i]=bin_conv[i]//10
    prob_dev.drop(col,axis=1,inplace=True)

for col in list(prob_report['Var']):
    applied = prob_report.loc[prob_report['Var']==col,'conversions'].values[0]
    if applied != '' and applied != 'categorical':
        try:
            for t in applied.split(','):
                prob_dev[col+'_'+t] = prob_dev[col].apply(transform_dict[t])
        except:
            pass
        
prob_dev.reset_index(inplace=True,drop=True)
prob_dev = prob_dev[['Symbol','Date','Open','Close']+list(prob_model_columns)]

development data for ADANIPORTS


C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://panda

development data for ASIANPAINT
development data for AXISBANK
development data for BAJAJ-AUTO
development data for BAJFINANCE
development data for BAJAJFINSV
development data for BHARTIARTL
development data for INFRATEL
development data for BPCL
development data for BRITANNIA
development data for CIPLA
development data for COALINDIA
development data for DRREDDY
development data for EICHERMOT
development data for GAIL
development data for GRASIM
development data for HCLTECH
development data for HDFC
development data for HDFCBANK
development data for HEROMOTOCO
development data for HINDALCO
development data for HINDUNILVR
development data for ICICIBANK
development data for IBULHSGFIN
development data for INDUSINDBK
development data for INFY
development data for IOC
development data for ITC
development data for JSWSTEEL
development data for KOTAKBANK
development data for LT
development data for M&M
development data for MARUTI
development data for NTPC
development data for ONGC
development

In [43]:
prob_dev.isnull().any()

Symbol                          False
Date                            False
Open                            False
Close                           False
Prev_Close_order5               False
High_1d_order5                  False
Low_1d_order5                   False
Last_1d_order5                  False
VWAP_1d_order5                  False
Volume_1d_order9                False
Turnover_1d_order11             False
Trades_1d_order7                False
Deliverable_Volume_1d_order9    False
pct_Deliverble_1d               False
Open_nifty_1d_order5            False
High_nifty_1d_order5            False
Low_nifty_1d_order5             False
Close_nifty_1d_order5           False
Volume_nifty_1d_order9          False
Turnover_nifty_1d_order11       False
Open_auto_1d_order5             False
High_auto_1d_order5             False
Low_auto_1d_order5              False
Close_auto_1d_order5            False
Open_bank_1d_order5             False
High_bank_1d_order5             False
Low_bank_1d_

In [44]:
x,x_lr,x_pca = np.array(price_dev[price_model_columns]),np.array(price_dev[price_lr_model_columns]),price_pca.transform(price_scale.transform(np.array(price_dev[price_model_columns])))
price_dev['Close_Prediction'] = (.48*price_lr.predict(x_lr)+.47*price_rf.predict(x_lr)+.48*price_gbm.predict(x_lr)+
                                 .47*price_rf_full.predict(x)+.47*price_gbm_full.predict(x)+.46*price_rf_pca.predict(x_pca)+
                                 .45*price_gbm_pca.predict(x_pca))/(.48*2+.47*3+.46+.45)
price_dev['Expected Return'] = (price_dev['Close_Prediction'] - price_dev['Prev_Close(in 10^5)']*(10**5))/(price_dev['Prev_Close(in 10^5)']*(10**5))
price_dev['Sharpe Ratio'] = price_dev['Expected Return']/price_dev['Voltality']

In [45]:
x,x_lr,x_pca = np.array(prob_dev[prob_model_columns]),np.array(prob_dev[prob_lr_model_columns]),prob_pca.transform(prob_scale.transform(np.array(prob_dev[prob_model_columns])))
prob_dev['Prob_Prediction'] = (prob_rf_full.predict_proba(x)[:,1]+prob_gbm_full.predict_proba(x)[:,1])/2

In [46]:
backtest_data = pd.merge(price_dev[['Date','Symbol','Open','Close','Prev_Close(in 10^5)','Close_Prediction','Expected Return','Voltality','Sharpe Ratio']],
                        prob_dev[['Date','Symbol','Prob_Prediction']],on=['Date','Symbol'])
backtest_data['Prev_Close'] = backtest_data['Prev_Close(in 10^5)']*(10**5)
backtest_data.drop('Prev_Close(in 10^5)',axis=1,inplace=True)
backtest_data = backtest_data[['Date','Symbol','Open','Close','Prev_Close','Close_Prediction','Expected Return','Voltality','Sharpe Ratio','Prob_Prediction']]
backtest_data['Sharpe Ratio'] = backtest_data['Expected Return']*backtest_data['Prob_Prediction']/backtest_data['Voltality']

In [47]:
dates = sorted(list(backtest_data['Date'].unique()))

In [105]:
value_list = []
first_day = backtest_data[backtest_data['Date']==dates[0]]
first_day.set_index('Symbol',inplace=True)
for comp in investments.index:
    value_list.append(investments.loc[comp,'shares']*first_day.loc[comp,'Open'])
investments['current value'] = value_list

actual_value_list = []
last_day = backtest_data[backtest_data['Date']==dates[-1]]
last_day.set_index('Symbol',inplace=True)
actual_portfolio.set_index('Symbol',inplace=True)
for comp in actual_portfolio.index:
    actual_value_list.append(actual_portfolio.loc[comp,'shares']*last_day.loc[comp,'Close'])
actual_portfolio['current value'] = actual_value_list
actual_portfolio.to_excel('actual_portfolio.xlsx')

In [106]:
append_list = [np.nan]
for company in [col for col in balance_sheet.columns if col not in ['Date','cash','portfolio_value']]:
    append_list.append(investments.loc[company,'current value'])
append_list+=[0,sum([i for i in append_list if not(np.isnan(i))])]
append_df = pd.DataFrame(append_list).transpose()
append_df.columns = balance_sheet.columns
balance_sheet = balance_sheet.append(append_df)

In [107]:
balance_sheet

,Date,ADANIPORTS,ASIANPAINT,AXISBANK,BAJAJ-AUTO,BAJFINANCE,BAJAJFINSV,BHARTIARTL,INFRATEL,BPCL,...,TECHM,TITAN,ULTRACEMCO,UPL,VEDL,WIPRO,YESBANK,ZEEL,cash,portfolio_value
0,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2019-04-02T00:00:00.000000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,50,50
2,2019-04-03T00:00:00.000000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,100,100
3,2019-04-04T00:00:00.000000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,150,150
4,2019-04-05T00:00:00.000000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,200,200
5,2019-04-08T00:00:00.000000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,250,250
6,2019-04-09T00:00:00.000000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,300,300
7,2019-04-10T00:00:00.000000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,350,350
8,2019-04-11T00:00:00.000000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,400,400
9,2019-04-12T00:00:00.000000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,450,450


In [108]:
backtest_data.columns

Index(['Date', 'Symbol', 'Open', 'Close', 'Prev_Close', 'Close_Prediction',
       'Expected Return', 'Voltality', 'Sharpe Ratio', 'Prob_Prediction'],
      dtype='object')

In [109]:
backtest_data.head()

,Date,Symbol,Open,Close,Prev_Close,Close_Prediction,Expected Return,Voltality,Sharpe Ratio,Prob_Prediction
0,2019-04-02,ADANIPORTS,383.0,379.70,383.05,343.058443,-0.104403,1.662118,-0.027322,0.434965
1,2019-04-03,ADANIPORTS,380.3,382.00,379.70,357.468394,-0.058550,1.665456,-0.016641,0.473362
2,2019-04-04,ADANIPORTS,382.0,380.20,382.00,353.923634,-0.073498,1.643409,-0.018802,0.420417
3,2019-04-05,ADANIPORTS,379.0,387.15,380.20,371.154509,-0.023791,0.736225,-0.013631,0.421812
4,2019-04-08,ADANIPORTS,388.3,379.90,387.15,357.390585,-0.076868,1.042389,-0.036144,0.490136


In [110]:
backtest_data.columns

Index(['Date', 'Symbol', 'Open', 'Close', 'Prev_Close', 'Close_Prediction',
       'Expected Return', 'Voltality', 'Sharpe Ratio', 'Prob_Prediction'],
      dtype='object')

In [111]:
daily_book = pd.DataFrame(columns=['Date', 'Symbol', 'Open', 'Close', 'Prev_Close', 'Close_Prediction',
       'Expected Return', 'Voltality', 'Sharpe Ratio', 'Prob_Prediction',
       'shares_before','shares_traded','shares_after','weights','investment_limit','investment_made','eod_value'])

In [112]:
money_in_hand = 0
daily_cash = cash/len(dates) 
for single_date in dates:
    daily_data = backtest_data[backtest_data['Date']==single_date]
    money_in_hand += daily_cash
    daily_data['weights'] = daily_data['Sharpe Ratio'].apply(lambda x: x if x>0 else 0)
    daily_data['weights'] = daily_data['weights']/daily_data['weights'].sum()
    daily_data['investment_limit'] = daily_data['weights']*money_in_hand
    daily_data['shares_before'] = 0
    for company in no_of_shares.keys():
        daily_data.loc[daily_data['Symbol']==company,'shares_before'] = no_of_shares[company]
    if single_date == dates[0]:
        daily_data['investment_made'] = daily_data['shares_before']*daily_data['Prev_Close']
        daily_data['eod_value'] = daily_data['shares_before']*daily_data['Open']
    else:
        daily_data['investment_made'] = 0
        daily_data['eod_value'] = 0
    
    shorts = list(daily_data.loc[(daily_data['Sharpe Ratio'] < 0) & (daily_data['shares_before'] > 0 ),'Symbol'])
    longs = list(daily_data.loc[(daily_data['Sharpe Ratio'] > 0),'Symbol'])
    
    daily_data['shares_traded'] = 0
    for company in longs:
        daily_data.loc[daily_data['Symbol']==company,'shares_traded'] = daily_data.loc[daily_data['Symbol']==company,
                                                                                       'investment_limit'].values[0]//daily_data.loc[
                                                                            daily_data['Symbol']==company,'Open'].values[0]
        if single_date != dates[0]:
            daily_data.loc[daily_data['Symbol']==company,'investment_made'] = daily_data.loc[daily_data['Symbol']==company,
                                                                                             'shares_traded'].values[0]*daily_data.loc[
                                                                                daily_data['Symbol']==company,'Open'].values[0]
        else:
            daily_data.loc[daily_data['Symbol']==company,'investment_made'] = daily_data.loc[daily_data['Symbol']==company,
                                                                                             'shares_traded'].values[0]*daily_data.loc[
                                                                                daily_data['Symbol']==company,'Open'].values[0]+daily_data.loc[daily_data['Symbol']==company,'investment_made'].values[0]
            
    for company in shorts:
        eod_before = daily_data.loc[daily_data['Symbol']==company,'shares_before'].values[0]*daily_data.loc[daily_data['Symbol']==company,
                                                                                                      'Prev_Close'].values[0]
        if single_date == dates[0]:
            investment_yet = daily_data.loc[daily_data['Symbol']==company,'investment_made'].values[0]
        else:
            investment_yet = daily_book.loc[daily_book['Symbol']==company,'investment_made'].sum()
        
        profits_before = float(eod_before - investment_yet)
        expected_loss = float(investment_yet*daily_data.loc[daily_data['Symbol']==company,'Expected Return'].values[0])
        
        if profits_before <= 0:
            daily_data.loc[daily_data['Symbol']==company,'shares_traded'] = -daily_data.loc[daily_data['Symbol']==company,'shares_before'].values[0]
        
        else:
            daily_data.loc[daily_data['Symbol']==company,'shares_traded'] = (max(-1,expected_loss/profits_before)*
                                                                                    investment_yet)//daily_data.loc[daily_data['Symbol']
                                                                                                                   ==company,'Open'].values[0]
        if single_date != dates[0]:
            daily_data.loc[daily_data['Symbol']==company,'investment_made'] = daily_data.loc[daily_data['Symbol']==company,
                                                                                             'shares_traded'].values[0]*daily_data.loc[
                                                                                daily_data['Symbol']==company,'Open'].values[0]
        else:
            daily_data.loc[daily_data['Symbol']==company,'investment_made'] = daily_data.loc[daily_data['Symbol']==company,
                                                                                             'shares_traded'].values[0]*daily_data.loc[
                                                                                daily_data['Symbol']==company,'Open'].values[0]+daily_data.loc[daily_data['Symbol']==company,'investment_made'].values[0]
            
    daily_data['shares_after']  = daily_data['shares_before'] + daily_data['shares_traded']
    
    daily_data['eod_value'] = daily_data['shares_after']*daily_data['Close']
    
    money_in_hand -= sum(list(daily_data['shares_traded']*daily_data['Open']))
    
    no_of_shares = {}
    for company in list(daily_data['Symbol']):
        no_of_shares[company] = daily_data.loc[daily_data['Symbol']==company,'shares_after'].values[0]
        
    daily_data = daily_data[daily_book.columns]
    daily_book = daily_book.append(daily_data)


C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.h

In [113]:
money_in_hand

1003.0000000000002

In [114]:
daily_book.columns

Index(['Date', 'Symbol', 'Open', 'Close', 'Prev_Close', 'Close_Prediction',
       'Expected Return', 'Voltality', 'Sharpe Ratio', 'Prob_Prediction',
       'shares_before', 'shares_traded', 'shares_after', 'weights',
       'investment_limit', 'investment_made', 'eod_value'],
      dtype='object')

In [115]:
daily_book['Profit_Ind'] = daily_book[['Open','Close']].apply(lambda x: 1 if x[1] > x[0] else 0,axis=1)
daily_book['Pred_Profit_Ind'] = daily_book[['Prev_Close','Close_Prediction']].apply(lambda x: 1 if x[1] > x[0] else 0,axis=1)
daily_book['Correct_Profit_Ind'] = daily_book[['Profit_Ind','Pred_Profit_Ind']].apply(lambda x: 1 if x[0]+x[1]==2 else 0,axis=1)

In [116]:
os.chdir('../'+key)

In [117]:
daily_book.to_excel('daily_book.xlsx',index=False)

In [118]:
daily_book.columns

Index(['Date', 'Symbol', 'Open', 'Close', 'Prev_Close', 'Close_Prediction',
       'Expected Return', 'Voltality', 'Sharpe Ratio', 'Prob_Prediction',
       'shares_before', 'shares_traded', 'shares_after', 'weights',
       'investment_limit', 'investment_made', 'eod_value', 'Profit_Ind',
       'Pred_Profit_Ind', 'Correct_Profit_Ind'],
      dtype='object')

In [119]:
cash = 0
for single_date in dates:
    daily_data = daily_book[daily_book['Date']==single_date]
    if single_date==dates[0]:
        cash+=daily_cash-(daily_data['investment_made'].sum() - list(balance_sheet['portfolio_value'])[-1])
    else:
        cash+=daily_cash-daily_data['investment_made'].sum()
    
    append_list = [single_date]
    for comp in [col for col in balance_sheet.columns if col not in ['Date','cash','portfolio_value']]:
        append_list.append(daily_data.loc[daily_data['Symbol']==comp,'eod_value'].values[0])
    append_list += [cash]
    append_list+= [sum(append_list[1:])]
    
    append_df = pd.DataFrame(append_list).transpose()
    append_df.columns = balance_sheet.columns
    balance_sheet = balance_sheet.append(append_df)
    

In [120]:
balance_sheet

,Date,ADANIPORTS,ASIANPAINT,AXISBANK,BAJAJ-AUTO,BAJFINANCE,BAJAJFINSV,BHARTIARTL,INFRATEL,BPCL,...,TECHM,TITAN,ULTRACEMCO,UPL,VEDL,WIPRO,YESBANK,ZEEL,cash,portfolio_value
0,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2019-04-02T00:00:00.000000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,50,50
2,2019-04-03T00:00:00.000000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,100,100
3,2019-04-04T00:00:00.000000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,150,150
4,2019-04-05T00:00:00.000000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,200,200
5,2019-04-08T00:00:00.000000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,250,250
6,2019-04-09T00:00:00.000000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,300,300
7,2019-04-10T00:00:00.000000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,350,350
8,2019-04-11T00:00:00.000000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,400,400
9,2019-04-12T00:00:00.000000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,450,450


In [121]:
balance_sheet.reset_index(inplace=True,drop=True)

In [122]:
os.chdir('../'+key)

In [123]:
balance_sheet.to_excel('balance_sheet.xlsx',index=False)

In [124]:
investments.columns

Index(['Money_invested', 'current value', 'shares'], dtype='object')

In [125]:
recommended_investments = pd.DataFrame()

In [126]:
recommended_investments['Symbol'] = list(daily_book['Symbol'].unique())

In [127]:
def profits(x):
    return daily_book.loc[(daily_book['Symbol']==x) & (daily_book['Date']==dates[-1]),'eod_value'].values[0]-daily_book.loc[(daily_book['Symbol']==x),
                                                                                                                           'investment_made'].sum()

In [128]:
recommended_investments['Profits'] = recommended_investments['Symbol'].apply(lambda x: profits(x))

In [129]:
recommended_investments['weights'] = recommended_investments['Profits'].apply(lambda x: x if x > 0 else 0)
recommended_investments['weights'] = recommended_investments['weights']/recommended_investments['weights'].sum()

In [130]:
recommended_investments

,Symbol,Profits,weights
0,ADANIPORTS,0.0,0.0
1,ASIANPAINT,0.0,0.0
2,AXISBANK,0.0,0.0
3,BAJAJ-AUTO,0.0,0.0
4,BAJFINANCE,0.0,0.0
5,BAJAJFINSV,0.0,0.0
6,BHARTIARTL,0.0,0.0
7,INFRATEL,0.0,0.0
8,BPCL,0.0,0.0
9,BRITANNIA,0.0,0.0


In [131]:
daily_book.columns

Index(['Date', 'Symbol', 'Open', 'Close', 'Prev_Close', 'Close_Prediction',
       'Expected Return', 'Voltality', 'Sharpe Ratio', 'Prob_Prediction',
       'shares_before', 'shares_traded', 'shares_after', 'weights',
       'investment_limit', 'investment_made', 'eod_value', 'Profit_Ind',
       'Pred_Profit_Ind', 'Correct_Profit_Ind'],
      dtype='object')

In [132]:
backtest_group = daily_book.groupby(by='Symbol').agg({'Profit_Ind' : ['sum'],'Pred_Profit_Ind':['sum'],
                                                     'Correct_Profit_Ind':['sum']})

In [133]:
backtest_group.head()

,Profit_Ind,Pred_Profit_Ind,Correct_Profit_Ind
,sum,sum,sum
Symbol,,,
ADANIPORTS,13,10,6
ASIANPAINT,8,1,0
AXISBANK,5,7,2
BAJAJ-AUTO,11,7,5
BAJAJFINSV,13,0,0


In [134]:
backtest_group.columns = backtest_group.columns.droplevel(1)

In [135]:
backtest_group.rename({'Profit_Ind':'#Rise_Event','Pred_Profit_Ind':'#Rise_Predicted','Correct_Profit_Ind':'#Actual_Rise'},
                      axis=1,inplace=True)

In [136]:
backtest_group.reset_index(inplace=True)

In [137]:
backtest_group['Precision'] = backtest_group['#Actual_Rise']/backtest_group['#Rise_Predicted']
backtest_group['Recall'] = backtest_group['#Actual_Rise']/backtest_group['#Rise_Event']
backtest_group['F1_Score'] = 2*backtest_group['Precision']*backtest_group['Recall']/(backtest_group['Precision']+backtest_group['Recall'])

In [138]:
backtest_group.fillna(0,inplace=True)

In [139]:
backtest_group

,Symbol,#Rise_Event,#Rise_Predicted,#Actual_Rise,Precision,Recall,F1_Score
0,ADANIPORTS,13,10,6,0.600000,0.461538,0.521739
1,ASIANPAINT,8,1,0,0.000000,0.000000,0.000000
2,AXISBANK,5,7,2,0.285714,0.400000,0.333333
3,BAJAJ-AUTO,11,7,5,0.714286,0.454545,0.555556
4,BAJAJFINSV,13,0,0,0.000000,0.000000,0.000000
5,BAJFINANCE,9,1,1,1.000000,0.111111,0.200000
6,BHARTIARTL,10,11,5,0.454545,0.500000,0.476190
7,BPCL,9,9,4,0.444444,0.444444,0.444444
8,BRITANNIA,6,7,1,0.142857,0.166667,0.153846
9,CIPLA,9,9,2,0.222222,0.222222,0.222222


In [140]:
total_portfolio_value = actual_portfolio['current value'].sum()+daily_cash*len(dates)

In [141]:
total_portfolio_value

1001.5

In [142]:
recommended_investments = pd.merge(recommended_investments,actual_portfolio[['Symbol','current value','shares']],on='Symbol')

KeyError: "['Symbol'] not in index"

In [ ]:
recommended_investments.rename({'shares':'shares_before'},axis=1,inplace=True)

In [ ]:
recommended_investments = pd.merge(recommended_investments,daily_book[daily_book['Date']==dates[-1]][['Symbol','Close']],
                                  on='Symbol')

In [ ]:
recommended_investments

In [ ]:
os.getcwd()

In [ ]:
recommended_investments = pd.merge(recommended_investments,graham_stats,on='Symbol')

In [ ]:
recommended_investments = pd.merge(recommended_investments,backtest_group,
                                  on='Symbol')

In [ ]:
recommended_investments.columns

In [ ]:
recommended_investments['weights'] = recommended_investments['weights']*recommended_investments['stability_score']*\
recommended_investments['leverage']*recommended_investments['F1_Score']*recommended_investments['roic']\
/(recommended_investments['P/E']*recommended_investments['Price/Book'])

recommended_investments['weights'] = recommended_investments['weights']/recommended_investments['weights'].sum()

In [ ]:
recommended_investments['Investment_Max'] = total_portfolio_value*recommended_investments['weights']

In [ ]:
recommended_investments['shares_traded'] = (recommended_investments['Investment_Max'] - recommended_investments['current value'])//\
recommended_investments['Close']

In [ ]:
recommended_investments['approx_investment_made'] = recommended_investments['shares_traded']*recommended_investments['Close']

In [ ]:
recommended_investments['shares_after'] = recommended_investments['shares_before'] + recommended_investments['shares_traded']

In [ ]:
recommended_investments.columns

In [ ]:
os.chdir('../'+key)

In [ ]:
os.getcwd()

In [ ]:
recommended_investments = recommended_investments[['Symbol','Profits','weights','P/E', 'Price/Book', 'Current assests', 
                                                   'Curent Liabilities','Long Term Debt','income','stability_score', 
                                                   'leverage','#Rise_Event',
       '#Rise_Predicted', '#Actual_Rise', 'Precision', 'Recall', 'F1_Score','roic','Close','current value','Investment_Max','approx_investment_made'
                                                   ,'shares_before','shares_traded','shares_after']]

In [ ]:
recommended_investments.to_excel('recommended_investments_details.xlsx',index=False)

In [ ]:
recommended_investments[['Symbol','Close','current value','Investment_Max','approx_investment_made'
                                                   ,'shares_before','shares_traded','shares_after']].\
to_excel('recommended_investments.xlsx',index=False)